In [13]:
import os
import pandas as pd
import numpy as np
import pyreadstat
import argparse
import re
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from sklearn.linear_model import LinearRegression
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.ensemble import ExtraTreesRegressor
import warnings
warnings.filterwarnings('ignore')


def johnson_relative_weights(X, y):
    """
    Реализация алгоритма Johnson's Relative Weights, максимально соответствующая SPSS
    
    Args:
        X (numpy.ndarray): Массив предикторов
        y (numpy.ndarray): Массив зависимой переменной
    
    Returns:
        dict: Словарь с результатами, включая R^2 и relative weights
    """
    import numpy as np
    from scipy import linalg
    from sklearn.linear_model import LinearRegression
    
    # Стандартизация переменных
    X_mean = np.mean(X, axis=0)
    X_std = np.std(X, axis=0, ddof=1)
    X_std_vals = (X - X_mean) / X_std
    
    y_mean = np.mean(y)
    y_std = np.std(y, ddof=1)
    y_std_vals = (y - y_mean) / y_std
    
    # Получаем корреляционную матрицу между предикторами
    RXX = np.corrcoef(X_std_vals, rowvar=False)
    
    # Получаем вектор корреляций между зависимой и независимыми переменными
    RXY = np.array([np.corrcoef(X_std_vals[:, i], y_std_vals)[0, 1] for i in range(X_std_vals.shape[1])])
    
    # Вычисляем стандартизованные бета-коэффициенты через OLS регрессию
    model = LinearRegression(fit_intercept=False)
    model.fit(X_std_vals, y_std_vals)
    beta = model.coef_
    
    # Правильный расчет R² для стандартизованных переменных
    r_squared = np.sum(beta * RXY)
    
    # Проверка значения R²
    if r_squared > 1.0:
        # Если R² получился больше 1, используем более надежный метод из sklearn
        r_squared = model.score(X_std_vals, y_std_vals)
    
    # Выполняем разложение корреляционной матрицы
    evals, evecs = linalg.eigh(RXX)
    
    # Обработка очень малых собственных значений для числовой стабильности
    epsilon = 1e-10
    evals[evals < epsilon] = epsilon
    
    # Создаем матрицу LAMBDA как в SPSS
    delta = np.sqrt(evals)
    LAMBDA = evecs @ np.diag(delta) @ evecs.T
    
    # Вычисляем квадрат LAMBDA
    LAMBDA_SQ = LAMBDA ** 2
    
    # Вычисляем веса предикторов для ортогональных переменных
    # В SPSS это: BETA = INV(LAMBDA) * RXY
    try:
        BETA_STAR = np.linalg.solve(LAMBDA, RXY)
    except np.linalg.LinAlgError:
        # Если матрица плохо обусловлена, используем псевдообратную матрицу
        BETA_STAR = np.linalg.lstsq(LAMBDA, RXY, rcond=None)[0]
    
    # Вычисляем "сырые" веса как в SPSS
    RAW_WEIGHTS = LAMBDA_SQ * (BETA_STAR ** 2)
    
    # Суммируем веса для каждого предиктора
    predictor_weights = np.sum(RAW_WEIGHTS, axis=1)
    
    # Нормализуем веса для получения процентного вклада
    if r_squared > epsilon:
        percentages = (predictor_weights / r_squared) * 100
    else:
        percentages = np.zeros_like(predictor_weights)
    
    return {
        'R2': r_squared,
        'rweights': predictor_weights,
        'percentages': percentages
    }

def calculate_johnson_weights(
    input_file: str,
    dependent_vars: list,
    independent_vars: list,
    subgroups: list = None,
    layer_var: str = None,
    min_sample_size: int = 100,
    output_dir: str = None
) -> str:
    """
    Calculate Johnson's Relative Weights for total sample and specified subgroups
    
    Args:
        input_file (str): Path to .sav file
        dependent_vars (list): List of dependent variables
        independent_vars (list): List of independent variables
        subgroups (list, optional): List of variables to create subgroups
        layer_var (str, optional): Variable to split subgroups by
        min_sample_size (int, optional): Minimum required sample size (default: 100)
        output_dir (str, optional): Directory to save results
        
    Returns:
        str: Path to saved Excel file with results
    """
    # Проверка путей на существование
    if not os.path.exists(input_file):
        print(f"Ошибка: Файл '{input_file}' не существует")
        return None
    
    # Проверка, что файл имеет расширение .sav
    if not input_file.lower().endswith('.sav'):
        print(f"Ошибка: Файл '{input_file}' должен иметь расширение .sav")
        return None
    
    # Проверка директории для сохранения результатов
    if output_dir:
        if not os.path.exists(output_dir):
            try:
                os.makedirs(output_dir)
                print(f"Создана директория: {output_dir}")
            except Exception as e:
                print(f"Ошибка при создании директории {output_dir}: {str(e)}")
                output_dir = os.path.dirname(input_file) or "."
                print(f"Результаты будут сохранены в директорию: {output_dir}")
    else:
        output_dir = os.path.dirname(input_file) or "."
    
    print(f"Анализ файла: {input_file}")
    print(f"Минимальный размер выборки: {min_sample_size}")
    
    # Чтение файла SPSS
    try:
        df, meta = pyreadstat.read_sav(input_file)
        print(f"Загружено {df.shape[0]} строк и {df.shape[1]} столбцов")
        
        # Выводим первые 10 переменных для проверки
        print("Первые 10 переменных в базе:")
        for i, col in enumerate(df.columns[:10]):
            print(f"  {i+1}. {col}")
        
        # Извлекаем метки значений для всех переменных
        value_labels = {}
        variables_with_labels = set(independent_vars + (subgroups or []))
        if layer_var:
            variables_with_labels.add(layer_var)
            
        for var in variables_with_labels:
            if var in meta.variable_value_labels:
                value_labels[var] = meta.variable_value_labels[var]
                print(f"\nМетки значений для переменной {var}:")
                for value, label in value_labels[var].items():
                    print(f"  {value}: {label}")
        
    except Exception as e:
        print(f"Ошибка при чтении файла: {str(e)}")
        return None
    
    # Проверка параметров
    if not dependent_vars:
        print("Ошибка: Не указаны зависимые переменные")
        return None
    
    if not independent_vars:
        print("Ошибка: Не указаны независимые переменные")
        return None
    
    # Проверка наличия всех необходимых переменных
    all_vars = dependent_vars + independent_vars
    if subgroups:
        all_vars.extend(subgroups)
    if layer_var:
        all_vars.append(layer_var)
    
    missing_vars = [var for var in all_vars if var not in df.columns]
    if missing_vars:
        print(f"Ошибка: В базе отсутствуют следующие переменные: {', '.join(missing_vars)}")
        print("Убедитесь, что вы указали правильные имена переменных.")
        return None
    
    # Создаем результирующий DataFrame для хранения всех результатов
    results = []
    
    # Создаем копию исходного датафрейма для импутации
    full_df = df.copy()
    
    # Функция для подготовки данных с единой импутацией
    def prepare_data_with_imputation(data, dep_var, subgroups=None, layer_var=None):
        """
        Modified imputation function to handle edge cases better
        """
        if data is None or len(data) == 0:
            print("Предупреждение: Пустой датафрейм, импутация невозможна")
            return data
            
        imputed_data = data.copy()
        
        # Replace both 98 and 99 with NaN in independent variables
        for var in independent_vars:
            imputed_data[var] = imputed_data[var].replace([98, 99], np.nan)
        
        # Check if imputation is needed
        if not imputed_data[independent_vars].isna().any().any():
            return imputed_data
            
        # Create imputer
        imputer = IterativeImputer(
            estimator=ExtraTreesRegressor(
                n_estimators=50,
                min_samples_leaf=10,
                max_features='sqrt'
            ),
            initial_strategy='median',
            max_iter=5,
            random_state=42,
            verbose=0
        )
        
        # Perform imputation on the entire dataset at once
        try:
            # Get variables with sufficient variance
            valid_vars = []
            for var in independent_vars:
                if imputed_data[var].nunique() > 1:
                    valid_vars.append(var)
            
            if not valid_vars:
                print("Нет переменных с достаточной вариацией для импутации")
                return imputed_data
            
            # Perform imputation
            X_imputed = imputer.fit_transform(imputed_data[valid_vars])
            
            # Update the original dataframe
            for i, var in enumerate(valid_vars):
                imputed_data[var] = X_imputed[:, i]
                
            return imputed_data
            
        except Exception as e:
            print(f"Ошибка при импутации: {str(e)}")
            return data
    
    # Проводим импутацию для всего датасета один раз с учетом структуры подгрупп
    full_df = prepare_data_with_imputation(df.copy(), None, subgroups, layer_var)
    
    # Функция для подготовки данных для конкретного анализа
    def prepare_analysis_data(data, dep_var):
        """
        Enhanced data preparation with better handling of missing and invalid values
        """
        # Copy needed variables
        working_df = data[independent_vars + [dep_var]].copy()
        
        # Print initial stats
        print(f"\nПодготовка данных для {dep_var}:")
        print(f"Исходное количество наблюдений: {len(working_df)}")
        
        # Remove invalid values in dependent variable
        working_df = working_df[
            (working_df[dep_var].notna()) & 
            (working_df[dep_var] != 99) &
            (working_df[dep_var] != 98)
        ]
        print(f"После удаления невалидных значений в зависимой переменной: {len(working_df)}")
        
        # Check remaining sample size
        if len(working_df) < min_sample_size:
            print(f"Недостаточно наблюдений после очистки (n={len(working_df)})")
            return None
        
        # Remove rows where ALL independent variables are missing
        working_df = working_df.dropna(subset=independent_vars, how='all')
        print(f"После удаления строк с полностью отсутствующими предикторами: {len(working_df)}")
        
        # Print summary statistics
        print("\nСтатистика по переменным:")
        print(f"Зависимая переменная ({dep_var}):")
        print(f"  Среднее: {working_df[dep_var].mean():.2f}")
        print(f"  Стд. откл.: {working_df[dep_var].std():.2f}")
        print(f"  Мин: {working_df[dep_var].min():.2f}")
        print(f"  Макс: {working_df[dep_var].max():.2f}")
        
        return working_df
    
    # Обновляем функцию calculate_weights для работы с новой структурой
    def calculate_weights(data, dep_var, group_info=None):
        """
        Enhanced weight calculation with better validation and debugging
        """
        if len(data) < min_sample_size:
            print(f"Недостаточно данных (n={len(data)}, требуется {min_sample_size})")
            return None
        
        try:
            analysis_data = prepare_analysis_data(data, dep_var)
            
            if analysis_data is None or len(analysis_data) < min_sample_size:
                return None
            
            # Convert to numpy arrays
            X = analysis_data[independent_vars].values
            y = analysis_data[dep_var].values
            
            # Print correlation information
            print("\nКорреляции с зависимой переменной:")
            for i, var in enumerate(independent_vars):
                corr = np.corrcoef(analysis_data[var].values, y)[0, 1]
                print(f"{var}: {corr:.4f}")
            
            # Check variable variance
            std_vars = np.std(X, axis=0)
            valid_indices = [i for i, std in enumerate(std_vars) if std > 0]
            
            if not valid_indices:
                print("Нет переменных с достаточной вариацией")
                return None
            
            X = X[:, valid_indices]
            valid_vars = [independent_vars[i] for i in valid_indices]
            
            # Calculate weights
            imp_results = johnson_relative_weights(X, y)
            
            print(f"\nРезультаты анализа:")
            print(f"R² = {imp_results['R2']:.4f}")
            print("Относительные веса:")
            for var, weight in zip(valid_vars, imp_results['rweights']):
                print(f"{var}: {weight:.4f}")
            
            # Create results dictionary
            weights_dict = {
                'Dependent Variable': dep_var,
                'Group Type': group_info['type'] if group_info else 'Total',
                'Group Variable': group_info['var'] if group_info else '',
                'Group Value': group_info['value'] if group_info else 'Total',
                'Group Value Label': value_labels.get(group_info['var'], {}).get(group_info['value'], '') if group_info and group_info['var'] else '',
                'Layer Variable': group_info['layer_var'] if group_info and 'layer_var' in group_info else '',
                'Layer Value': group_info['layer_value'] if group_info and 'layer_value' in group_info else '',
                'Layer Value Label': value_labels.get(group_info['layer_var'], {}).get(group_info['layer_value'], '') if group_info and 'layer_var' in group_info else '',
                'Sample Size': len(analysis_data),
                'R-squared': imp_results['R2']
            }
            
            # Add weights and percentages
            for i, var in enumerate(valid_vars):
                weights_dict[f'Weight_{var}'] = imp_results['rweights'][i]
                weights_dict[f'Percentage_{var}'] = imp_results['percentages'][i]
            
            return weights_dict
            
        except Exception as e:
            print(f"Ошибка при расчете весов: {str(e)}")
            import traceback
            traceback.print_exc()
            return None
    
    # Для каждой зависимой переменной
    for dependent_var in dependent_vars:
        print(f"\nАнализ для зависимой переменной: {dependent_var}")
        
        # Расчет для общей выборки
        print("  Расчет для общей выборки")
        total_results = calculate_weights(full_df, dependent_var)
        if total_results:
            results.append(total_results)
        
        # Расчет для подгрупп
        if subgroups:
            for subgroup_var in subgroups:
                print(f"\n  Анализ подгруппы: {subgroup_var}")
                subgroup_values = df[subgroup_var].dropna().unique()
                
                for subgroup_value in subgroup_values:
                    print(f"    Значение: {subgroup_value}")
                    
                    # Расчет для подгруппы
                    subgroup_df = full_df[full_df[subgroup_var] == subgroup_value].copy()
                    
                    subgroup_info = {
                        'type': 'Subgroup',
                        'var': subgroup_var,
                        'value': subgroup_value
                    }
                    
                    subgroup_results = calculate_weights(subgroup_df, dependent_var, subgroup_info)
                    if subgroup_results:
                        results.append(subgroup_results)
                    
                    # Если есть layer_var, делаем дополнительное разбиение
                    if layer_var:
                        print(f"      Разбиение по слою: {layer_var}")
                        layer_values = df[df[subgroup_var] == subgroup_value][layer_var].dropna().unique()
                        
                        for layer_value in layer_values:
                            print(f"        Значение слоя: {layer_value}")
                            layer_df = full_df[
                                (full_df[subgroup_var] == subgroup_value) & 
                                (full_df[layer_var] == layer_value)
                            ].copy()
                            
                            layer_info = {
                                'type': 'Layer',
                                'var': subgroup_var,
                                'value': subgroup_value,
                                'layer_var': layer_var,
                                'layer_value': layer_value,
                                'layer_value_label': value_labels.get(layer_var, {}).get(layer_value, '')
                            }
                            
                            layer_results = calculate_weights(layer_df, dependent_var, layer_info)
                            if layer_results:
                                results.append(layer_results)
    
    # Проверка результатов
    if not results:
        print("\nНе было получено результатов. Проверьте параметры анализа и данные.")
        return None
    
    # Создаем DataFrame с результатами
    results_df = pd.DataFrame(results)
    
    # Определяем путь для сохранения результатов
    if output_dir is None:
        output_dir = os.path.dirname(input_file) or "."
    
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    # Формируем имя файла
    timestamp = pd.Timestamp.now().strftime('%Y%m%d_%H%M%S')
    analysis_type = 'layered' if layer_var else 'basic'
    output_file = os.path.join(output_dir, f"johnson_weights_{timestamp}_{analysis_type}.xlsx")
    
    try:
        # Создаем Excel-файл
        wb = Workbook()
        ws = wb.active
        ws.title = "Johnson's Relative Weights"
        
        # Получаем все столбцы, но только те, которые есть в DataFrame
        base_cols = ['Dependent Variable', 'Group Type', 'Group Variable', 'Group Value']
        
        # Add Layer columns only if they exist in the results
        if 'Layer Variable' in results_df.columns and 'Layer Value' in results_df.columns:
            base_cols.extend(['Layer Variable', 'Layer Value', 'Layer Value Label'])
        
        # Add remaining standard columns
        base_cols.extend(['Sample Size', 'R-squared'])
        
        # Filter base_cols to include only columns that exist in results_df
        base_cols = [col for col in base_cols if col in results_df.columns]
        
        # Get weight and percentage columns
        weight_cols = [col for col in results_df.columns if col.startswith('Weight_')]
        pct_cols = [col for col in results_df.columns if col.startswith('Percentage_')]
        
        # Combine all columns that exist in the DataFrame
        ordered_cols = base_cols + weight_cols + pct_cols
        results_df = results_df[ordered_cols]
        
        # Транспонируем данные для Excel
        headers = list(results_df.columns)
        
        # Добавляем заголовки как первую колонку
        for i, header in enumerate(headers):
            ws.cell(row=i+1, column=1, value=header)
        
        # Добавляем данные
        for i, row_idx in enumerate(range(len(results_df))):
            row = results_df.iloc[row_idx]
            for j, value in enumerate(row):
                ws.cell(row=j+1, column=i+2, value=value)
        
        # Автонастройка ширины столбцов
        for column in ws.columns:
            max_length = 0
            column_letter = column[0].column_letter
            for cell in column:
                try:
                    if len(str(cell.value)) > max_length:
                        max_length = len(str(cell.value))
                except:
                    pass
            adjusted_width = (max_length + 2)
            ws.column_dimensions[column_letter].width = adjusted_width
        
        # Сохраняем файлы
        wb.save(output_file)
        
        # Save CSV file
        csv_file = output_file.replace('.xlsx', '.csv')
        results_df.to_csv(csv_file, index=False)
        
        # Single message block
        print(f"\nРезультаты сохранены в файлы:")
        print(f"- Excel: {os.path.basename(output_file)}")
        print(f"- CSV: {os.path.basename(csv_file)}")
        print(f"Директория: {os.path.dirname(output_file)}")
        
        return output_file
        
    except Exception as e:
        print(f"Ошибка при сохранении в Excel: {str(e)}")
        try:
            csv_file = output_file.replace('.xlsx', '.csv')
            results_df.to_csv(csv_file, index=False)
            print(f"Результаты сохранены только в CSV: {os.path.basename(csv_file)}")
            print(f"Директория: {os.path.dirname(csv_file)}")
            return csv_file
        except:
            print("Не удалось сохранить результаты.")
            return None
    


In [14]:
# Определяем переменные

input_file = '/Users/jbaukova/Documents/Projects/!NPS-2025/MR-3943 Поиск w1/!Результаты/финал/aggregated/Scen6-G_aggr.sav'
dependent_vars = ['satisfaction']
independent_vars = [
    "product_attr_1",
    "product_attr_2",
    "product_attr_3",
    "product_attr_4", 
    #"product_attr_12",
    #"product_attr_13",
    #"product_attr_61",
    #"product_attr_62",
    #"product_attr_63",
    #"product_attr_64",
    "brand_attr_1", 
    #"brand_attr_2",
    #"brand_attr_3",
    #"brand_attr_4",
]

subgroups = ['fin_services_type']
layer_var = 'brand_id'
min_sample_size = 100

output_dir = '/Users/jbaukova/Documents/Projects/!NPS-2025/MR-3943 Поиск w1/!Результаты/финал/Johnson/test'


# Вызываем функцию расчета

output_file = calculate_johnson_weights(
    input_file=input_file,
    dependent_vars=dependent_vars,
    independent_vars=independent_vars,
    subgroups=subgroups,
    layer_var=layer_var,
    min_sample_size=100,
    output_dir=output_dir
)


Анализ файла: /Users/jbaukova/Documents/Projects/!NPS-2025/MR-3943 Поиск w1/!Результаты/финал/aggregated/Scen6-G_aggr.sav
Минимальный размер выборки: 100
Загружено 9075 строк и 74 столбцов
Первые 10 переменных в базе:
  1. id
  2. Qpanel
  3. QSearch7
  4. QSearch8
  5. QNeuro1
  6. QNeuro2_r3
  7. QNeuro2_r4
  8. Search_seg
  9. weight_fin
  10. Check_Search_1

Анализ для зависимой переменной: satisfaction
  Расчет для общей выборки

Подготовка данных для satisfaction:
Исходное количество наблюдений: 9075
После удаления невалидных значений в зависимой переменной: 2715
После удаления строк с полностью отсутствующими предикторами: 2715

Статистика по переменным:
Зависимая переменная (satisfaction):
  Среднее: 8.23
  Стд. откл.: 2.20
  Мин: 1.00
  Макс: 10.00

Корреляции с зависимой переменной:
product_attr_1: 0.6446
product_attr_2: 0.6830
product_attr_3: 0.6511
product_attr_4: 0.4542
brand_attr_1: 0.5647

Результаты анализа:
R² = 0.5260
Относительные веса:
product_attr_1: 0.1138
product

In [15]:
# Определяем переменные

input_file = '/Users/jbaukova/Documents/Projects/!NPS-2025/MR-3943 Поиск w1/!Результаты/финал/aggregated/Scen6-G_aggr.sav'
dependent_vars = ['satisfaction']
independent_vars = [
    "product_attr_1",
    "product_attr_2",
    "product_attr_3",
    "product_attr_4", 
    #"product_attr_12",
    #"product_attr_13",
    #"product_attr_61",
    #"product_attr_62",
    #"product_attr_63",
    #"product_attr_64",
    "brand_attr_1", 
    #"brand_attr_2",
    #"brand_attr_3",
    #"brand_attr_4",
]

subgroups = ['brand_id']
layer_var = None
min_sample_size = 100

output_dir = '/Users/jbaukova/Documents/Projects/!NPS-2025/MR-3943 Поиск w1/!Результаты/финал/Johnson/test'


# Вызываем функцию расчета

output_file = calculate_johnson_weights(
    input_file=input_file,
    dependent_vars=dependent_vars,
    independent_vars=independent_vars,
    subgroups=subgroups,
    layer_var=layer_var,
    min_sample_size=100,
    output_dir=output_dir
)


Анализ файла: /Users/jbaukova/Documents/Projects/!NPS-2025/MR-3943 Поиск w1/!Результаты/финал/aggregated/Scen6-G_aggr.sav
Минимальный размер выборки: 100
Загружено 9075 строк и 74 столбцов
Первые 10 переменных в базе:
  1. id
  2. Qpanel
  3. QSearch7
  4. QSearch8
  5. QNeuro1
  6. QNeuro2_r3
  7. QNeuro2_r4
  8. Search_seg
  9. weight_fin
  10. Check_Search_1

Анализ для зависимой переменной: satisfaction
  Расчет для общей выборки

Подготовка данных для satisfaction:
Исходное количество наблюдений: 9075
После удаления невалидных значений в зависимой переменной: 2715
После удаления строк с полностью отсутствующими предикторами: 2715

Статистика по переменным:
Зависимая переменная (satisfaction):
  Среднее: 8.23
  Стд. откл.: 2.20
  Мин: 1.00
  Макс: 10.00

Корреляции с зависимой переменной:
product_attr_1: 0.6442
product_attr_2: 0.6829
product_attr_3: 0.6504
product_attr_4: 0.4538
brand_attr_1: 0.5650

Результаты анализа:
R² = 0.5258
Относительные веса:
product_attr_1: 0.1137
product